In [4]:
!pip install openai yfinance pandas numpy matplotlib seaborn requests beautifulsoup4

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from investment_recommendation_system import InvestmentRecommendationSystem

OPENAI_API_KEY = "open-ai-api-key"
system = InvestmentRecommendationSystem(OPENAI_API_KEY)

print("Hệ thống đã được khởi tạo thành công!")

Hệ thống đã được khởi tạo thành công!


In [10]:
sample_news_1 = """
Tesla đã công bố kế hoạch mở rộng sản xuất pin lithium-ion và xây dựng 
thêm 5 nhà máy sản xuất xe điện tại Mỹ. Động thái này được kỳ vọng sẽ 
thúc đẩy toàn bộ ngành công nghiệp xe điện và các công ty cung cấp linh kiện."""

print("=== KIỂM TRA VỚI TIN TỨC VỀ XE ĐIỆN ===")
recommendations_1 = system.get_stock_recommendations(sample_news_1)
print("Kết quả khuyến nghị:")
print(recommendations_1)

=== KIỂM TRA VỚI TIN TỨC VỀ XE ĐIỆN ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.444755 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.961967 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:investment_recommendation_system:Lỗi khi gọi OpenAI API: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Kết quả khuyến nghị:
{'status': 'error', 'message': "Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"}


In [ ]:
sample_news_2 = """
FDA đã phê duyệt một loại thuốc điều trị Alzheimer mới, mở ra cơ hội lớn 
cho các công ty dược phẩm. Thị trường thuốc điều trị bệnh thần kinh được 
dự báo sẽ tăng trưởng mạnh trong 5 năm tới.
"""

print("\n=== KIỂM TRA VỚI TIN TỨC VỀ DƯỢC PHẨM ===")
recommendations_2 = system.get_stock_recommendations(sample_news_2)
print("Kết quả khuyến nghị:")
print(recommendations_2)

In [ ]:
policy_news = """
Chính phủ công bố gói kích thích kinh tế 2 tỷ USD tập trung vào 
phát triển hạ tầng công nghệ, bao gồm 5G, AI và điện toán đám mây.
Các công ty công nghệ được kỳ vọng sẽ hưởng lợi từ chính sách này.
"""

print("\n=== KIỂM TRA KHUYẾN NGHỊ NGÀNH ===")
sector_recommendations = system.get_sector_recommendations(policy_news)
print("Kết quả khuyến nghị ngành:")
print(sector_recommendations)

In [ ]:
if recommendations_1.get('status') == 'yes':
    print("\n=== KIỂM TRA TÍNH HỢP LỆ CỦA CỔ PHIẾU ===")
    validation_1 = system.validate_stock_recommendations(recommendations_1['recommendations'])
    
    print(f"Tổng số khuyến nghị: {validation_1['total_recommendations']}")
    print(f"Số cổ phiếu hợp lệ: {validation_1['valid_stocks']}")
    print(f"Tỷ lệ hợp lệ: {validation_1['valid_stocks']/validation_1['total_recommendations']*100:.1f}%")
    
    print("\nChi tiết từng cổ phiếu:")
    for detail in validation_1['validation_details']:
        print(f"- {detail['symbol']}: {'Hợp lệ' if detail['exists'] else 'Không hợp lệ'}")
        if detail['exists']:
            print(f"  Giá hiện tại: ${detail['current_price']:.2f}")
            print(f"  Ngành: {detail.get('sector', 'N/A')}")


In [ ]:
if recommendations_1.get('status') == 'yes':
    print("\n=== KIỂM TRA HIỆU SUẤT (BACKTEST) ===")
    backtest_1 = system.backtest_recommendations(recommendations_1['recommendations'], days_forward=30)
    
    if 'average_return' in backtest_1:
        print(f"Lợi nhuận trung bình sau 30 ngày: {backtest_1['average_return']:.2f}%")
        print(f"Tỷ lệ thắng: {backtest_1['win_rate']*100:.1f}%")
        
        # Vẽ biểu đồ kết quả
        valid_results = [r for r in backtest_1['individual_results'] if 'returns_pct' in r]
        if valid_results:
            returns = [r['returns_pct'] for r in valid_results]
            symbols = [r['symbol'] for r in valid_results]
            
            plt.figure(figsize=(10, 6))
            bars = plt.bar(symbols, returns, color=['green' if r > 0 else 'red' for r in returns])
            plt.title('Hiệu suất các cổ phiếu được khuyến nghị (30 ngày)')
            plt.xlabel('Mã cổ phiếu')
            plt.ylabel('Lợi nhuận (%)')
            plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
            
            # Thêm nhãn giá trị
            for bar, return_val in zip(bars, returns):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                        f'{return_val:.1f}%', ha='center', va='bottom')
            
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()


In [ ]:
def compare_with_market(recommendations, market_symbol="SPY", days=30):
    """
    So sánh hiệu suất khuyến nghị với chỉ số thị trường
    """
    print(f"\n=== SO SÁNH VỚI CHỈ SỐ THỊ TRƯỜNG ({market_symbol}) ===")
    
    # Lấy hiệu suất thị trường
    import yfinance as yf
    market = yf.Ticker(market_symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days + 30)
    market_hist = market.history(start=start_date, end=end_date)
    
    if len(market_hist) >= days:
        market_start = market_hist['Close'].iloc[0]
        market_end = market_hist['Close'].iloc[days]
        market_return = (market_end - market_start) / market_start * 100
        
        print(f"Hiệu suất thị trường ({market_symbol}): {market_return:.2f}%")
        
        # So sánh với khuyến nghị
        if recommendations.get('status') == 'yes':
            backtest = system.backtest_recommendations(recommendations['recommendations'], days_forward=days)
            if 'average_return' in backtest:
                portfolio_return = backtest['average_return']
                print(f"Hiệu suất trung bình portfolio: {portfolio_return:.2f}%")
                print(f"Chênh lệch so với thị trường: {portfolio_return - market_return:.2f}%")
                
                if portfolio_return > market_return:
                    print("✅ Portfolio vượt trội so với thị trường")
                else:
                    print("❌ Portfolio kém hơn so với thị trường")

# Chạy so sánh
if recommendations_1.get('status') == 'yes':
    compare_with_market(recommendations_1)


In [ ]:
def analyze_by_sector(recommendations):
    """
    Phân tích khuyến nghị theo ngành
    """
    if recommendations.get('status') != 'yes':
        return
    
    print("\n=== PHÂN TÍCH THEO NGÀNH ===")
    
    sectors = {}
    for rec in recommendations['recommendations']:
        symbol = rec['symbol']
        try:
            import yfinance as yf
            stock = yf.Ticker(symbol)
            info = stock.info
            sector = info.get('sector', 'Unknown')
            
            if sector not in sectors:
                sectors[sector] = []
            sectors[sector].append(symbol)
        except:
            continue
    
    print("Phân bố theo ngành:")
    for sector, symbols in sectors.items():
        print(f"- {sector}: {', '.join(symbols)}")
    
    # Vẽ biểu đồ
    if sectors:
        plt.figure(figsize=(10, 6))
        sector_counts = [len(symbols) for symbols in sectors.values()]
        plt.pie(sector_counts, labels=sectors.keys(), autopct='%1.1f%%')
        plt.title('Phân bố khuyến nghị theo ngành')
        plt.axis('equal')
        plt.show()

# Chạy phân tích ngành
if recommendations_1.get('status') == 'yes':
    analyze_by_sector(recommendations)